### NMODL Kinetic Scheme

Some definitions of reaction kinetics & mass action laws that are relevant for how `KINETIC` blocks are translated into a system of ODEs in NMODL

#### Reaction Kinetics

- We consider a set of reaction species $A_j$, with corresponding Molar concentrations $Y_j$
- They react according to a set of reaction equations:

$$\sum_j \nu_{ij}^L A_j \overset{k_i}{\rightarrow} \sum_j \nu_{ij}^R A_j$$

where
- $k_i$ is the rate coefficient
- $\nu_{ij}^L$, $\nu_{ij}^R$ are stochiometric coefficients - must be positive integers (including zero)

***
#### Law of Mass Action

- This allows us to convert these reaction equations to a set of ODEs

$$\frac{dY_j}{dt} = \sum_i \Delta \nu_{ij} r_i$$

where $\Delta \nu_{ij} = \nu_{ij}^R - \nu_{ij}^L$, and
$$r_i = k_i \prod_j Y_j^{\nu_{ij}^R}$$
***

#### KINETIC block format

A reaction equation is specifed in the mod file as
```
~ A0 + 3A1 + 2A2 + ... <-> 2A0 + A1 + ... (kf, kb)
```
where
- `A0` etc are the species $A_j$
- the integer preceeding a species (with or without a space) is the corresponding stochiometric coefficient $\nu_{ij}$ (by default 1 if not present)
- `kf` is the forwards reaction rate $k^{(f)}_j$
- `kb` is the backwards reaction rate $k^{(b)}_j$, i.e. the reaction rate for the same reaction with the LHS and RHS exchanged

***
We can convert these statements to a system of ODEs:
$$\frac{dY_j}{dt} = \sum_i \Delta \nu_{ij} (r^{(f)}_i - r^{(b)}_i)$$

where $\Delta \nu_{ij} = \nu_{ij}^R - \nu_{ij}^L$, and
$$
\begin{aligned}
r^{(f)}_i &= k^{(f)}_i \prod_j Y_j^{\nu_{ij}^{L}}\\
r^{(b)}_i &= k^{(b)}_i \prod_j Y_j^{\nu_{ij}^{R}}
\end{aligned}
$$
***

#### Example

Given the following KINETIC statement
```
~ h <-> m (a,b)
```
We have 2 state variables, and 1 reaction statement, so 
- $i = \{0\}$
- $j = \{0, 1\}$

i.e. 

- the state vector $Y$ has 2 elements
$$
Y = \left(
\begin{aligned}
h \\
m 
\end{aligned}
\right)
$$

- the stochiometric coefficients are 1x2 matrices
$$
\nu^L = \left(
\begin{aligned}
1 && 0
\end{aligned}
\right)
$$
$$
\nu^R = \left(
\begin{aligned}
0 && 1
\end{aligned}
\right)
$$

- the rate vectors contain 1 element:
$$
k^{(f)} = a
$$
$$
k^{(b)} = b
$$

Using these we can construct the forwards and blackwards fluxes:
$$
r^{(f)} = a h
$$
$$
r^{(b)} = b m
$$

and finally we find the ODEs in matrix form:
$$
\frac{dY}{dt} =
\left(
\begin{aligned}
-1 && 1
\end{aligned}
\right)
(ah - bm)
$$

which in terms of the state variables can be written:
$$
\begin{aligned}
\frac{dh}{dt} &= bm - ah \\
\frac{dm}{dt} &= ah - bm
\end{aligned}
$$
***

#### Other types of reaction statement

Can also have a reaction statement of the form
```
~ h << (a)
```
where the LHS must be a state variable, and the RHS is an expression inside parentheses.

The meaning of this statement is to add `a` to the differential equation for `h`, i.e.
$$
\frac{dh}{dt} += a
$$

***
Finally there is a statement of the form
```
x + 2y + ... -> (a)
```
which is a one-way reaction statement with no reaction products.

This is just syntactic sugar for a special case of the standard `<->` reaction equation, where the backwards rate is set to zero and there are no states on the RHS of the reaction, so the above is equivalent to
```
x + 2y + ... <-> (a, 0)
```
***

#### [TODO] CONSERVE

Converse statement allows specification of a conservation law, e.g.

```
CONSERVE h + m = 0
```
This can then be used to eliminate one of these variables from the set of ODEs, for example by removing the dmdt equation, and instead calculating m in terms of the algebraic relation above.

***

#### [TODO] fflux / bflux

Directly after each kinetic statement, NEURON provides the fflux and bflux variable that can be referenced inside the mod file (!). Note that it always refers to the fflux/bflux from the preceeding kinetic statement.

So we should parse all non-kinetic statements in the mod file following a kinetic statement (until we get to a new kinetic statement), and replace any fflux or bflux variables with corresponding expression.

***

#### [TODO] inlining

The usual issues about inlining function calls, e.g. when the rate is a function call.

In principle also allowed for it to be a function of state variables, in which case vital to inline this information for the ODEs (although we could also just say that this is not supported, as not necessary for the user to write the mod file in this way, state variable dependence can always be written explicitly).

***